In [75]:
from copy import deepcopy
from abc import ABC, abstractmethod, abstractproperty
from typing import List, Generator, Set, Optional, Tuple
from collections import defaultdict

class Point:
    def __init__(self, x, y):
        self.x: int = x
        self.y: int = y

class Rock(ABC):
    def __init__(self, ref : Point):
        self.ref = ref
        self.prev_ref = None
        
    @abstractmethod
    def rocks_positions(self) -> Set[Tuple]:
        raise NotImplementedError()
    
    @abstractproperty
    def width(self) -> int:
        raise NotImplementedError()
    
    @abstractproperty
    def rock_type(self) -> str:
        raise NotImplementedError()
    
    def move_left(self):
        self.prev_ref = Point(self.ref.x, self.ref.y)
        self.ref.x -= 1
    
    def move_right(self):
        self.prev_ref = Point(self.ref.x, self.ref.y)
        self.ref.x += 1
        
    def move_up(self):
        self.prev_ref = Point(self.ref.x, self.ref.y)
        self.ref.y += 1
        
    def move_down(self):
        self.prev_ref = Point(self.ref.x, self.ref.y)
        self.ref.y -= 1
        
    def restore_prev(self):
        self.ref = Point(self.prev_ref.x, self.prev_ref.y)
    
class LRock(Rock):
    """
    ..#
    ..#
    ###
    """
    def rocks_positions(self):
        return ((self.ref.x, self.ref.y),
                (self.ref.x+1, self.ref.y),
                (self.ref.x+2, self.ref.y),
                (self.ref.x+2, self.ref.y+1),
                (self.ref.x+2, self.ref.y+2))
        
    @property
    def width(self) -> int:
        return 3
    
    @property
    def rock_type(self) -> str:
        return "lrock"
    

class HBarRock(Rock):
    """
    ####
    """
    def rocks_positions(self):
        return ((self.ref.x, self.ref.y),
                (self.ref.x+1, self.ref.y),
                (self.ref.x+2, self.ref.y),
                (self.ref.x+3, self.ref.y))
    
    @property
    def width(self) -> int:
        return 4
    
    @property
    def rock_type(self) -> str:
        return "hbarrock"
        
class VBarRock(Rock):
    """
    #
    #
    #
    #
    """
    def rocks_positions(self):
        return ((self.ref.x, self.ref.y),
                (self.ref.x, self.ref.y+1),
                (self.ref.x, self.ref.y+2),
                (self.ref.x, self.ref.y+3))
        
    @property
    def width(self) -> int:
        return 1
        
    @property
    def rock_type(self) -> str:
        return "vbarrock"
    
class CrossRock(Rock):
    """
    .#.
    ###
    .#.
    """
    def rocks_positions(self):
        return ((self.ref.x+1, self.ref.y),
                (self.ref.x, self.ref.y+1),
                (self.ref.x+1, self.ref.y+1),
                (self.ref.x+2, self.ref.y+1),
                (self.ref.x+1, self.ref.y+2))
        
    @property
    def width(self) -> int:
        return 3
    
    @property
    def rock_type(self) -> str:
        return "crossrock"
    
class SquareRock(Rock):
    """
    ##
    ##
    """
    def rocks_positions(self) -> Set[Tuple]:
        return {(self.ref.x, self.ref.y),
                (self.ref.x+1, self.ref.y),
                (self.ref.x, self.ref.y+1),
                (self.ref.x+1, self.ref.y+1)}
    
    @property
    def width(self) -> int:
        return 2
    
    @property
    def rock_type(self) -> str:
        return "squarerock"
    
class RockFactory:
    def __init__(self):
        self.rock_types = [HBarRock, CrossRock, LRock, VBarRock, SquareRock]
        self.current_rock_type = iter(self.rock_types)
        
    def next_rock(self, ref : Point) -> Generator[Rock, None, None]:
        while True:
            try:
                yield next(self.current_rock_type)(ref)
            except StopIteration:
                self.current_rock_type = iter(self.rock_types)

class GameBoard:
    def __init__(self, width: int):
        self.width: int = width
        self.max_widths = [-1]*self.width
        self.all_positions = set()
        
    def is_outside_board(self, rock: Rock) -> bool:
        return rock.ref.x < 0 or (rock.ref.x + rock.width) > self.width or rock.ref.y < 0

    def is_rock_blocked(self, rock: Rock) -> bool:
        if self.is_outside_board(rock):
            return True
        for p in rock.rocks_positions():
            if self.max_widths[p[0]] >= p[1]:
                return True
        return False
    
    @property
    def height(self):
        return max(self.max_widths) + 1
    
    def add_rock(self, rock: Rock):
        rocks_positions = rock.rocks_positions()
        for p in rocks_positions:
            self.max_widths[p[0]] = max(self.max_widths[p[0]], p[1])

class Moves:
    def __init__(self, path):
        self.moves = []
        with open(path) as f:
            for l in f:
                l = l.strip('\n')
                for c in l:
                    self.moves.append(c)
        self.iter = iter(self.moves)
    
    def next_move(self):
        while True:
            try:
                yield next(self.iter)
            except StopIteration:
                self.iter = iter(self.moves)

def main_func():
    factory = RockFactory()
    current_rock : Optional[Rock] = None
    nb_rocks = 0
    #max_rocks = 1000000000000
    max_rocks = 2022
    board = GameBoard(7)
    move_rock = True
    player_moves = Moves('../day17.txt')

    while True:
        if not current_rock or board.is_rock_blocked(current_rock):
            if current_rock:                
                current_rock.restore_prev()
                board.add_rock(current_rock)
                nb_rocks += 1
                if nb_rocks == max_rocks:
                    break
                
            current_rock = next(factory.next_rock(Point(2, board.height+3)))
            
            move_rock = True
        elif move_rock:
            next_move = next(player_moves.next_move())
            if next_move == '>':
                current_rock.move_right()
            elif next_move == '<':
                current_rock.move_left()
            if board.is_rock_blocked(current_rock):
                current_rock.restore_prev()
            move_rock = False
        else:
            current_rock.move_down()
            move_rock = True
            
    print(board.height)

# importing library
import cProfile
import pstats
import io
from pstats import SortKey
 
# Creating profile object
ob = cProfile.Profile()
ob.enable()

main_func()

ob.disable()
sec = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(ob, stream=sec).sort_stats(sortby)
ps.print_stats()
 
print(sec.getvalue())

3111
         9296188 function calls (9281890 primitive calls) in 2.224 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    2.224    1.112 /home/bmaisonneuve/adventofcode2022/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000    2.224    1.112 {built-in method builtins.exec}
        1    0.000    0.000    2.224    2.224 /tmp/ipykernel_326/2847399884.py:254(<module>)
        1    0.044    0.044    2.224    2.224 /tmp/ipykernel_326/2847399884.py:207(main_func)
     2023    0.004    0.000    1.976    0.001 /tmp/ipykernel_326/2847399884.py:179(height)
     2022    0.783    0.000    1.971    0.001 {built-in method builtins.max}
  9001539    1.188    0.000    1.188    0.000 /tmp/ipykernel_326/2847399884.py:182(<genexpr>)
    37635    0.044    0.000    0.125    0.000 /tmp/ipykernel_326/2847399884.py:169(is_rock_blocked)
    37635    0.031    0.00

In [48]:
l = [1,2,3,4,5]
it = iter(l)
while True:
    next(it)

StopIteration: 

In [11]:
for _ in range(100):
    print('Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)')

Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goûter (banane et un gâteau à la confiture de fraise)
Celeste a mangé sont goût

In [13]:
while True:
    print('Que veux-tu pour le goûter?')
    s = input()
    print(f'Tiens voilà une {s}')

Que veux-tu pour le goûter?
Tiens voilà une banane
Que veux-tu pour le goûter?
Tiens voilà une pomme
Que veux-tu pour le goûter?


KeyboardInterrupt: Interrupted by user